# University Project II - Big Data

Author : Ophiase

In [72]:
ENABLE_DOWNLOAD = False
ENABLE_UNZIP = False
ENABLE_COMPUTE_ZIP_TO_PARQUET = False

### Dependencies

In [73]:
import shutil
import os
import requests
import pandas as pd
import re
import pyspark.sql.functions as F
import zipfile
from functools import reduce
import math

In [74]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import matplotlib.pyplot as plt
import pyspark
import logging

logging.getLogger("pyspark").setLevel(logging.ERROR)
# os.environ["PYSPARK_SUBMIT_ARGS"] = "--driver-memory 2g"

import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import when, mean, stddev, skewness, kurtosis, expr, date_format
import pyspark.sql.functions as pf
from pyspark.sql.functions import col, lit, udf, dayofweek, avg, year, month, unix_timestamp
from pyspark.sql.types import DoubleType


import altair as alt
import plotly
import plotly.express as px

import scipy
from scipy.stats import skew, kurtosis

import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [75]:
spark = SparkSession.builder \
    .appName("Big Data Project") \
    .config("spark.driver.memory", "3g") \
    .config("spark.sql.files.maxPartitionBytes", "2g") \
    .getOrCreate()

## Download the DATA

In [5]:
# from 2014 to 2023

zip_files = []

def download_data():
    trip_urls = [
        (year, f"https://s3.amazonaws.com/tripdata/{year}-citibike-tripdata.zip")
        for year in range(2014, 2023 + 1)
    ]

    if not os.path.exists(os.path.join('data')) :
        os.makedirs('data')

    for year, url in trip_urls:
        basename = os.path.join('data', str(year) + "_" + 'citibike_tripdata')
        zip_filename = basename + ".zip"
        csv_filename = basename + ".csv"
        zip_files.append((year, zip_filename, csv_filename))

        if not ENABLE_DOWNLOAD : continue
        print(f'Check {basename} ...')

        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024
        progress = 0

        if not os.path.exists(zip_filename) and not os.path.exists(csv_filename) :
            with open(zip_filename, 'wb') as f:
                for data in response.iter_content(block_size):
                    if data:
                        f.write(data)
                        progress += len(data)
                        print(f'\rDownloaded {progress}/{total_size} bytes', end='')

            print(f'\nDownload complete: {zip_filename}')

    print("Finished")

download_data()

Finished


In [6]:
def unzip_data():
    for (year, zip_filename, csv_filename) in zip_files:
        # if year < 2018: continue # WARNING : DISABLE THIS LINE

        if not zipfile.is_zipfile(zip_filename):
            print("Corrupted zip file.")
            break

        if os.path.exists("tmp"):
            shutil.rmtree("tmp")

        print("Unzip : ", zip_filename)
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall("tmp")
        print("Process ..")

        # find the folder in tmp
        items = os.listdir("tmp")
        for folder in items :
            if not os.path.isdir(os.path.join("tmp", folder)) or \
                folder.startswith("__") :
                continue
            
            # find all the folder in this folder
            sub_folders = os.listdir(os.path.join("tmp", folder))
            for sub_folder in sub_folders :
                if not os.path.isdir(os.path.join("tmp", folder, sub_folder)) or \
                    sub_folder.startswith(".") : 
                    continue
                
                sub_item = os.listdir(os.path.join("tmp", folder, sub_folder))
                for leaf in sub_item :
                    # move the csv inside to data
                    from_path = os.path.join("tmp", folder, sub_folder, leaf)
                    dest_path = os.path.join("data", leaf)
                    if os.path.exists(dest_path) : 
                        os.remove(dest_path)

                    shutil.move(from_path, "data")

    if os.path.exists("tmp"):
        shutil.rmtree("tmp")

if ENABLE_UNZIP :
    unzip_data()

To optimize disk usage, we could have unziped one file at a time and convert its content instantaneously to `.parquet`.

## Convert the Data

### Raw Analysis

In [7]:
csv_reader = spark.read.option("header", "true") \
            .option("inferSchema", "true").csv

In [8]:
def find_all_csv():
    all_csv = []
    for item in os.listdir("data"):
        if not item.endswith(".csv") :
            continue
        all_csv.append(item)
    return sorted(all_csv)

all_csv = find_all_csv()

if False: # check column_names.txt
    for item in all_csv:    
        df = csv_reader(os.path.join("data", item))
        print(f"item {item} : {df.columns}")

By looking at the previous code output *(cached in `column_names.txt`)*, \
we notice the following columns between 2014-01 $\to$ 2021-01 (included) :
- `['tripduration', 'starttime', 'stoptime', 'start station id', 'start station name', `\
`'start station latitude', 'start station longitude', 'end station id', 'end station name', `\
`'end station latitude', 'end station longitude', 'bikeid', 'usertype', 'birth year', 'gender']`
    - The naming convention is not exactly the same between : `201610-citibike-tripdata_1.csv` $\to$ `201703-citibike-tripdata.csv_1.csv` : \
    `['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',` \
    `'Start Station Name', 'Start Station Latitude', 'Start Station Longitude',` \
    `'End Station ID', 'End Station Name', 'End Station Latitude',` \
    `'End Station Longitude', 'Bike ID', 'User Type', 'Birth Year', 'Gender']`

The columns change between 2021-02 $\to$ 2023-12 (included) :
- `['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', `\
`'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', `\
`'start_lng', 'end_lat', 'end_lng', 'member_casual']`

We decide the following matching (**O.** as before 2021-02, **N.** as after 2021-02):

- `O.'tripduration'` as function : `stoptime` - `startime`
    - Renamed as `trip_duration`

- `N.'started_at'` $\leftarrow$ `O.'starttime'`
- `N.'ended_at'` $\leftarrow$ `O.'stoptime'` 
- `N.'start_station_id'` $\leftarrow$ `O.'start station id'`
    - type : string
- `N.'start_station_name'` $\leftarrow$ `O.'start station name'`
- `N.'start_lat'` $\leftarrow$ `O.'start station latitude'`
- `N.'start_lng'` $\leftarrow$ `O.'start station longitude'`
- `N.'end_station_id'` $\leftarrow$ `O.'end station id'`
    - type : string
- `N.'end_station_name'` $\leftarrow$ `O.'end station name'`
- `N.'end_lat'` $\leftarrow$ `O.'end station latitude'`
- `N.'end_lng'` $\leftarrow$ `O.'end station longitude'`
`
- `N.'ride_id'` $\leftarrow$ `O.'bikeid'` (format is not the same)
    - Both type of ID can registered as string

- `N.'member_casual'` $\leftarrow$ `O.'usertype'` (format is not the same)
    - Mapping : `O.Subscriber`, `O.Customer` $\to$ `N.member`, `N.casual`

- `O.'birth year` : (None) for elements of N
    - Renamed as `birth_year`
- `O.'gender'` : (None) for elements of N
- `N.'rideable_type'` : (None) for elements of O

- We will also add a binary column `old_format` to indicate if the data comes from `O` or `N` as defined above.

In [9]:
col_mapping_1 = {
    'tripduration': 'trip_duration',
    'usertype': 'member_casual',
    'birth year': 'birth_year',

    'starttime': 'started_at',
    'stoptime': 'ended_at',
    'start station id': 'start_station_id',
    'start station name': 'start_station_name',
    'start station latitude': 'start_lat',
    'start station longitude': 'start_lng',
    'end station id': 'end_station_id',
    'end station name': 'end_station_name',
    'end station latitude': 'end_lat',
    'end station longitude': 'end_lng',
    'bikeid': 'ride_id',
}

col_mapping_2 = {
    'Trip Duration': 'tripduration',
    'Start Time': 'starttime',
    'Stop Time': 'stoptime',
    
    'Start Station ID': 'start station id',
    'Start Station Name': 'start station name',
    'Start Station Latitude': 'start station latitude',
    'Start Station Longitude': 'start station longitude',

    'End Station ID': 'end station id',
    'End Station Name' : 'end station name',
    'End Station Latitude' : 'end station latitude',
    'End Station Longitude' : 'end station longitude',
    
    'Bike ID' : 'bikeid',
    'User Type' : 'usertype',
    'Birth Year' : 'birth year',
    'Gender' : 'gender'
}


In [10]:
def check_unique_values(df, column):
    return df.select(column).dropDuplicates().rdd.map(lambda row: row[0]).collect()

In [11]:
def fast_check():
    df_o = spark.read.csv(os.path.join("data", all_csv[0]), header=True, inferSchema=True)
    df_o = df_o.select(
        [col(old_col).alias(col_mapping_1.get(old_col, old_col)) for old_col in df_o.columns]
        )

    df_n = spark.read.csv(os.path.join("data", all_csv[-1]), header=True, inferSchema=True)
    
    print(check_unique_values(df_o, "member_casual"))
    print(check_unique_values(df_o, "gender"))
    print(check_unique_values(df_n, "rideable_type"))

    df_o.printSchema()
    df_n.printSchema()
    
    # df.show()

fast_check()

['Subscriber', 'Customer']
[1, 2, 0]
['electric_bike', 'classic_bike']
root
 |-- trip_duration: integer (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_lat: double (nullable = true)
 |-- start_lng: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_lat: double (nullable = true)
 |-- end_lng: double (nullable = true)
 |-- ride_id: integer (nullable = true)
 |-- member_casual: string (nullable = true)
 |-- birth_year: string (nullable = true)
 |-- gender: integer (nullable = true)

root
 |-- ride_id: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: string (nullable = 

### File format Selection : Parquet

<img src="resources/file-formats.png" alt="Drawing" style="width: 400px;"/>

Our [course](https://stephane-v-boucheron.fr/slides/tbd/slides06_file-formats.html#/title-slide) on Big Data file formats.

Parquet suits our needs for the project:

- Suitable for laptop execution
- 37GB dataset size manageable with Parquet's compression
- Supports splitability for processing subsets of data on minimal configuration
- Compatible with Apache Spark

### CSV $\to$ Parquet

In [12]:
if not os.path.exists(os.path.join('computed')) :
    os.makedirs('computed')

In [13]:
factorial_columns = ["member_casual", "gender", "rideable_type"]

In [58]:
# Lower Case O. part

# add missing columns
def csv_o_process(df):
    df = df.select(
        [col(old_col).alias(col_mapping_1.get(old_col, old_col)) for old_col in df.columns])

    df = df.withColumn("birth_year", 
                       when(col("birth_year") == r"\N", lit(None))
                       .otherwise(col("birth_year"))
                       .cast("integer")
                       )

    df = df.withColumn("start_station_id", col("start_station_id").cast("string"))\
            .withColumn("end_station_id", col("end_station_id").cast("string")) \
            .withColumn("ended_at", col("ended_at").cast("timestamp")) \
            .withColumn("started_at", col("started_at").cast("timestamp")) \
            .withColumn("ride_id", col("ride_id").cast("string"))

    df = df.withColumn("member_casual",
                        when(col("member_casual") == "Subscriber", lit("member")) \
                        .otherwise(lit("casual")))
    
    df = df.withColumn("rideable_type", lit(None).cast('string'))
    df = df.withColumn("old_format", lit(True))
    
    df = df.withColumn("gender", 
         when(col('gender') == 0, lit(None) ) \
        .when(col('gender') == 1, lit("Male")) \
        .when(col('gender') == 2, lit("Female")).cast("string"))

    df = df.select(*sorted(df.columns))

    # df.write.mode("append").parquet(parquet_file)
    return df

def csv_to_parquet_part_1(csv_file) :
    df = spark.read.csv(csv_file, header=True, inferSchema=True)
    return csv_o_process(df)

In [59]:
# Upper case O. part

def csv_to_parquet_part_2(csv_file) :
    df = spark.read.csv(csv_file, header=True, inferSchema=True)
    
    df = df.select(
        [col(old_col).alias(col_mapping_2.get(old_col, old_col)) for old_col in df.columns])
    
    return csv_o_process(df)

In [60]:
# N. Part

def csv_to_parquet_part_3(csv_file) :
    df = spark.read.csv(csv_file, header=True, inferSchema=True)

    # df = df.withColumn("trip_duration", 
    #     (unix_timestamp(col("ended_at")) - unix_timestamp(col("started_at"))) \
    #     .cast("integer"))
    
    df = df.withColumn("trip_duration", 
        F.when(
            (col("started_at").isNotNull()) & (col("ended_at").isNotNull()) & (col("ended_at") > col("started_at")),
            (F.unix_timestamp(col("ended_at")) - F.unix_timestamp(col("started_at"))).cast("integer")
        ).otherwise(None))
    
    df = df.withColumn("old_format", lit(False))
    df = df.withColumn("birth_year", lit(None).cast("integer"))

    df = df.withColumn("gender", lit(None).cast("string"))

    df = df.withColumn("start_station_id", col("start_station_id").cast("string"))\
            .withColumn("end_station_id", col("end_station_id").cast("string")) \
            .withColumn("ended_at", col("ended_at").cast("timestamp")) \
            .withColumn("started_at", col("started_at").cast("timestamp")) \
            .withColumn("ride_id", col("ride_id").cast("string"))

    df = df.select(*sorted(df.columns))

    # df.write.mode("append").parquet(parquet_file)
    return df


#### File Partitioning

We decide to partition columns on `"year(started_at)"`, `"month(started_at)"`, `"start_station_id"`

Even with Lazy Loading of the CSV file, dataset doesn't seems to fit in RAM, thus we partition by hand the .parquet file intos multiples files

In [ ]:
# Scheduler

schedule = [
    ("201401-citibike-tripdata_1.csv", 1),
    ("201610-citibike-tripdata_1.csv", 2),
    ("201704-citibike-tripdata.csv_1.csv", 1),
    ("202102-citibike-tripdata_1.csv", 3)
]

def process_csv_files(csv_files, schedule):
    schedule_pointer = 0
    df_buffer = []

    for index, csv_file in enumerate(csv_files):
        csv_path = os.path.join("data", csv_file)
        
        if index % 20 == 0 :
            print(f"[{schedule_pointer}, {index}] {csv_file}")
        if (schedule_pointer < len(schedule) - 1) and (csv_file == schedule[schedule_pointer + 1][0]) :
            schedule_pointer += 1
            print(f"{csv_file} | {schedule_pointer} : {schedule[schedule_pointer]}")

        schedule_mode = schedule[schedule_pointer][1]
        target_function = [csv_to_parquet_part_1, csv_to_parquet_part_2, csv_to_parquet_part_3][schedule_mode - 1]

        df_to_add = target_function(csv_path)
        df_buffer.append(df_to_add)

    print ("REDUCE STEP")
    
    df = reduce(lambda df1, df2: df1.union(df2), df_buffer)
    df = df.withColumn("year",
        when(col("started_at").isNotNull(), 
                year(col("started_at")).cast("integer")
            ).otherwise(lit(None).cast("integer"))
    )

    print ("WITH year/month")

    parquet_path = os.path.join(
        'computed', 'precompiled', f'citibike.parquet')

    df \
        .partitionBy("year") \
        .mode("overwrite").parquet(parquet_path)
        # .write.option("maxRecordsPerFile", 70000000) \

if ENABLE_COMPUTE_ZIP_TO_PARQUET :
    process_csv_files(all_csv, schedule)

### Verifying Parquet is working

In [1]:
df = spark.read.parquet(os.path.join("computed", "precompiled_ter", "*.parquet"))

print("Number of rows : " + str(df.count()))
df.show()

NameError: name 'spark' is not defined

### Dimension tables

In [91]:
stations_dim = df.select("start_station_id", "start_station_name", 
                                "start_lat", "start_lng") \
                        .withColumnRenamed("start_station_id", "station_id") \
                        .withColumnRenamed("start_station_name", "station_name") \
                        .withColumnRenamed("start_lat", "lat") \
                        .withColumnRenamed("start_lng", "lng")

stations_dim = stations_dim.unionByName(
    df.select("end_station_id", "end_station_name", 
                    "end_lat", "end_lng") \
             .withColumnRenamed("end_station_id", "station_id") \
             .withColumnRenamed("end_station_name", "station_name") \
             .withColumnRenamed("end_lat", "lat") \
             .withColumnRenamed("end_lng", "lng")
            ).distinct()

                #, allowMissingColumns=True

stations_dim.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[station_id#592593, station_name#592598, lat#592603, lng#592608], functions=[])
   +- Exchange hashpartitioning(station_id#592593, station_name#592598, lat#592603, lng#592608, 200), ENSURE_REQUIREMENTS, [plan_id=65605]
      +- HashAggregate(keys=[station_id#592593, station_name#592598, knownfloatingpointnormalized(normalizenanandzero(lat#592603)) AS lat#592603, knownfloatingpointnormalized(normalizenanandzero(lng#592608)) AS lng#592608], functions=[])
         +- Union
            :- Project [start_station_id#591492 AS station_id#592593, start_station_name#591493 AS station_name#592598, start_lat#591490 AS lat#592603, start_lng#591491 AS lng#592608]
            :  +- FileScan parquet [start_lat#591490,start_lng#591491,start_station_id#591492,start_station_name#591493,year#591496] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/home/astragaliton/Documents/documentat

In [92]:
stations_dim.write \
            .mode("overwrite") \
            .parquet(os.path.join("computed", "stations_dim", "stations_dim.parquet"))

#.option("maxRecordsPerFile", 70000000) \

24/05/22 03:35:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/22 03:35:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/22 03:35:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/22 03:35:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/22 03:35:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/22 03:35:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/astragaliton/anaconda3/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 

24/05/22 03:36:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/22 03:36:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/22 03:36:26 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/22 03:36:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/22 03:36:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/22 03:36:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


## Analysis

In [53]:
end_date_pre_covid = '2020-02-29'
start_date_post_covid = '2020-03-01'

df_pre_covid = df.filter(col('ended_at') <= end_date_pre_covid)
df_2018 = df.filter((col('ended_at') <= '2019-01-01') & (col('started_at') > '2018-01-01')) 

df_post_covid = df.filter(col('started_at') >= start_date_post_covid)
df_2022 = df.filter((col('ended_at') <= '2023-01-01') & (col('started_at') > '2022-01-01')) 

In [54]:
print("Before Covid-19")
df_pre_covid.show(5)

Before Covid-19


+----------+-----------+------------+--------------+--------------------+--------------------+------+-------------+----------+-------+-------------+------------------+------------------+----------------+--------------------+--------------------+-------------+
|birth_year|    end_lat|     end_lng|end_station_id|    end_station_name|            ended_at|gender|member_casual|old_format|ride_id|rideable_type|         start_lat|         start_lng|start_station_id|  start_station_name|          started_at|trip_duration|
+----------+-----------+------------+--------------+--------------------+--------------------+------+-------------+----------+-------+-------------+------------------+------------------+----------------+--------------------+--------------------+-------------+
|      1993|40.71850211|-73.98329859|         349.0|Rivington St & Ri...|2018-12-20 14:12:...|  Male|       member|      true|  34649|         NULL|       40.72580614|      -73.97422494|           339.0|  Avenue D & E 12

In [55]:
print("After Covid-19")
df_post_covid.show(5)

After Covid-19
+----------+-----------------+-----------------+--------------+--------------------+-------------------+------+-------------+----------+----------------+-------------+-----------------+------------------+----------------+--------------------+-------------------+-------------+
|birth_year|          end_lat|          end_lng|end_station_id|    end_station_name|           ended_at|gender|member_casual|old_format|         ride_id|rideable_type|        start_lat|         start_lng|start_station_id|  start_station_name|         started_at|trip_duration|
+----------+-----------------+-----------------+--------------+--------------------+-------------------+------+-------------+----------+----------------+-------------+-----------------+------------------+----------------+--------------------+-------------------+-------------+
|      NULL|40.72970805644994|-73.9865979552269|       5746.02|     E 10 St & 2 Ave|2023-09-03 10:24:16|  NULL|       member|     false|B0A0F1DEFA4B72FC|e

#### Compute distances

In [56]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Rayon de la Terre en kilomètres
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

haversine_udf = udf(haversine, DoubleType())

In [57]:
def compte_trip_distance(df, verbose=False):
    if verbose : print("compute distance")

    df = df.filter(
        col("start_lat").isNotNull() & 
        col("start_lng").isNotNull() & 
        col("end_lat").isNotNull() & 
        col("end_lng").isNotNull()
    )

    df = df.withColumn(
        "distance_km", haversine_udf(col("start_lat"), 
                                     col("start_lng"), 
                                     col("end_lat"), 
                                     col("end_lng")))
    
    return df

df_2018 = compte_trip_distance(df_2018)
df_2022 = compte_trip_distance(df_2022)
df_post_covid = compte_trip_distance(df_post_covid)

### Trip distance by day

In [58]:
def compute_trip_distance_over_week(df, verbose=False):
    '''
        Returns in the pandas format
    '''
    if verbose : print("compute day of week")
    df = df.withColumn("day_of_the_week", dayofweek(col("started_at")))

    if verbose : print("group by")
    
    avg_distance_by_day_of_the_week = df.groupBy("day_of_the_week") \
        .agg(avg("distance_km").alias("avg_distance_km"))
    
    avg_distance_by_day_of_the_week = avg_distance_by_day_of_the_week.withColumn(
        "day_of_the_week",
        when(col("day_of_the_week") == 1, "Sunday")
        .when(col("day_of_the_week") == 2, "Monday")
        .when(col("day_of_the_week") == 3, "Tuesday")
        .when(col("day_of_the_week") == 4, "Wednesday")
        .when(col("day_of_the_week") == 5, "Thursday")
        .when(col("day_of_the_week") == 6, "Friday")
        .when(col("day_of_the_week") == 7, "Saturday")
    )

    return avg_distance_by_day_of_the_week.toPandas()

In [59]:
trip_distance_over_week_2018 = compute_trip_distance_over_week(df_2018)
trip_distance_over_week_2022 = compute_trip_distance_over_week(df_2022)

In [60]:
def visualize_trip_distance_over_week(df_1, df_2, description_1="", description_2=""):
    def specific_chart(df, description_bonus):
        min_value = df['avg_distance_km'].min() * 0.99
        max_value = df['avg_distance_km'].max() * 1.02

        return alt.Chart(df).mark_bar().encode(
                x=alt.X('day_of_the_week:O', title='Day', sort=None),
                y=alt.Y('avg_distance_km:Q', title='Average distance (km)',
                        scale=alt.Scale(domain=(min_value, max_value))),
                color=alt.Color('day_of_the_week:N', title='Day of The Week'),
                tooltip=['avg_distance_km', 'day_of_the_week']
            ).properties(
                title= 'Average trip distance per Day' + description_bonus,
                width= 300,
                height= 200
            )
        
    chart = alt.hconcat(
        specific_chart(df_1, description_1),
        specific_chart(df_2, description_2)
    ).configure_axis(
        labelAngle=45
    )

    chart = chart.configure_axis(
        titleFontSize=12,
        labelFontSize=10,
        labelOverlap='parity'
    ).configure_legend(
        titleFontSize=12,
        labelFontSize=10
    )

    return chart

visualize_trip_distance_over_week(
    trip_distance_over_week_2018, trip_distance_over_week_2022, 
    "(2018)", "(2022)"
    ).interactive()

alt.HConcatChart(...)

####  Number of trips for each pickup/dropoff location couple

#### Trip distance distribution for gender

In [61]:
def compute_trip_distance_over_genders(df, verbose=False):
    '''
        Returns in Pandas format
    '''
    df = df.filter(col("gender").isNotNull())
    
    avg_distance = df.groupBy("gender") \
        .agg(avg("distance_km").alias("avg_distance_km"))
    
    return avg_distance.toPandas()

In [67]:
trip_distance_over_genders_2018 = compute_trip_distance_over_genders(df_2018)
trip_distance_over_genders_post_covid = compute_trip_distance_over_genders(df_post_covid)

In [68]:
def visualize_trip_distance_over_genders(df_1, df_2, description_1="", description_2=""):
    def specific_chart(df, description_bonus):
        min_value = 0 # df['avg_distance_km'].min() * 0.99
        max_value = df['avg_distance_km'].max() * 1.02

        return alt.Chart(df).mark_bar().encode(
                x=alt.X('gender:O', title='Gender', sort=None),
                y=alt.Y('avg_distance_km:Q', title='Average distance (km)',
                        scale=alt.Scale(domain=(min_value, max_value))),
                color=alt.Color('gender', title='Gender'),
                tooltip=['avg_distance_km', 'gender']
            ).properties(
                title= 'Average trip distance per Gender' + description_bonus,
                width= 300,
                height= 200
            )
        
    chart = alt.hconcat(
        specific_chart(df_1, description_1),
        specific_chart(df_2, description_2)
    ).configure_axis(
        labelAngle=45
    )

    chart = chart.configure_axis(
        titleFontSize=12,
        labelFontSize=10,
        labelOverlap='parity'
    ).configure_legend(
        titleFontSize=12,
        labelFontSize=10
    )

    return chart

visualize_trip_distance_over_genders(
    trip_distance_over_genders_2018, trip_distance_over_genders_post_covid,
    " (2018)", " (post covid)"
    ).interactive()


alt.HConcatChart(...)

#### Trip distance distribution for age ranges

In [69]:
def compute_trip_distance_over_ages(df, verbose=False):
    '''
        Returns in Pandas format
    '''
        
    df = df.filter(col("birth_year").isNotNull())
    df = df.withColumn("age", year(col('started_at')) - col('birth_year'))

    df = df.withColumn(
        "age_range", 
         when(col('age') <= 14, lit(None) ) \
        .when((15 <= col('age')) & (col('age') <= 24), lit('15-24')) \
        .when((25 <= col('age')) & (col('age') <= 44), lit('25-45')) \
        .when((45 <= col('age')) & (col('age') <= 54), lit('45-54')) \
        .when((55 <= col('age')) & (col('age') <= 64), lit('55-64')) \
        .when(65 <= col('age'), lit('65+')) \
    )

    avg_distance = df.groupBy("age_range") \
        .agg(avg("distance_km").alias("avg_distance_km"))

    return avg_distance.toPandas()

In [70]:
trip_distance_over_ages_2018 = compute_trip_distance_over_ages(df_2018)
trip_distance_over_ages_post_covid = compute_trip_distance_over_ages(df_post_covid)

In [73]:
def visualize_trip_distance_over_ages(df_1, df_2, description_1="", description_2=""):
    def specific_chart(df, description_bonus):
        min_value = 0 # df['avg_distance_km'].min() * 0.80
        max_value = df['avg_distance_km'].max() * 1.02

        return alt.Chart(df).mark_bar().encode(
                x=alt.X('age_range:O', title='Age Range', sort=None),
                y=alt.Y('avg_distance_km:Q', title='Average distance (km)',
                        scale=alt.Scale(domain=(min_value, max_value))),
                color=alt.Color('age_range', title='Age Range'),
                tooltip=['avg_distance_km', 'age_range']
            ).properties(
                title= 'Average trip distance per age range' + description_bonus,
                width= 300,
                height= 200
            )
        
    chart = alt.hconcat(
        specific_chart(df_1, description_1),
        specific_chart(df_2, description_2)
    ).configure_axis(
        labelAngle=45
    )

    chart = chart.configure_axis(
        titleFontSize=12,
        labelFontSize=10,
        labelOverlap='parity'
    ).configure_legend(
        titleFontSize=12,
        labelFontSize=10
    )

    return chart

visualize_trip_distance_over_ages(
    trip_distance_over_ages_2018, trip_distance_over_ages_post_covid,
    " (2018) ", " (post covid) "
    ).interactive()


alt.HConcatChart(...)

#### Trip distance distribution for different kind of bikes

In [ ]:
def compute_trip_distance_over_bikes(df, verbose=False):
    df = df.filter(col("rideable_type").isNotNull())
    
    avg_distance = df.groupBy("rideable_type") \
        .agg(avg("distance_km").alias("avg_distance_km"))

    return avg_distance.toPandas

trip_distance_over_bikes_2022 = compute_trip_distance_over_bikes(df_2022)

In [ ]:
def visualize_trip_distance_over_bikes(df_1):
    min_value = 0 # df['avg_distance_km'].min() * 0.80
    max_value = df['avg_distance_km'].max() * 1.02

    chart = alt.Chart(df).mark_bar().encode(
            x=alt.X('rideable_type:O', title='Age Range', sort=None),
            y=alt.Y('avg_distance_km:Q', title='Average distance (km)',
                    scale=alt.Scale(domain=(min_value, max_value))),
            color=alt.Color('rideable_type', title='Type of bike'),
            tooltip=['avg_distance_km', 'age_range']
        ).properties(
            title= 'Average trip distance per type of types of bike after covid',
            width= 300,
            height= 200
        )
        
    chart = chart.configure_axis(
        labelAngle=45,
        titleFontSize=12,
        labelFontSize=10,
        labelOverlap='parity'
    ).configure_legend(
        titleFontSize=12,
        labelFontSize=10
    )

    return chart

visualize_trip_distance_over_bikes(
    trip_distance_over_bikes_post_covid,
    ).interactive()


## Hour of the week

### The number of pickups/docks

### The average distance

### The average trip duration

### The average number of ongoing trips

## Monitoring

In [78]:
df.explain(True)

== Parsed Logical Plan ==
Relation [birth_year#591479,end_lat#591480,end_lng#591481,end_station_id#591482,end_station_name#591483,ended_at#591484,gender#591485,member_casual#591486,old_format#591487,ride_id#591488,rideable_type#591489,start_lat#591490,start_lng#591491,start_station_id#591492,start_station_name#591493,started_at#591494,trip_duration#591495,year#591496] parquet

== Analyzed Logical Plan ==
birth_year: int, end_lat: double, end_lng: double, end_station_id: string, end_station_name: string, ended_at: timestamp, gender: string, member_casual: string, old_format: boolean, ride_id: string, rideable_type: string, start_lat: double, start_lng: double, start_station_id: string, start_station_name: string, started_at: timestamp, trip_duration: int, year: int
Relation [birth_year#591479,end_lat#591480,end_lng#591481,end_station_id#591482,end_station_name#591483,ended_at#591484,gender#591485,member_casual#591486,old_format#591487,ride_id#591488,rideable_type#591489,start_lat#591490

2] Relation doesn't differ between Analyzed and Optimized Logical Plan. The query optimizer in the RDBMS did not find any opportunities to further optimize the logical plan after the initial analysis phase.


3] The Physical Plan differs from the Optimized Logical Plan in that it includes the ColumnarToRow operator, which is a Spark-specific operator that converts data from the columnar format used for storage and processing to the row format used for execution. The FileScan parquet operator is also specific to Spark and is used to scan Parquet data files.

4] TODO The provided code snippet does not include any information about stages or the Spark UI, so I cannot determine how many stages are necessary to complete the Spark job. HashAggregate and Exchange hashpartitioning are used for aggregation and shuffling data between partitions, respectively, but their roles cannot be inferred from the given information.

5] TODO: The Physical Plan shows a single FileScan parquet operator, which suggests that no shuffle operations are performed. However, without more context, I cannot definitively say whether shuffle operations are performed elsewhere in the query plan.

6] TODO: In Spark, tasks are the individual units of execution within a stage. The number of tasks in a stage depends on the number of partitions of the input data. However, without information about the input data and partitioning, I cannot determine the number of tasks in the stages.

## Spatial Informations